## Library import

In [1]:
import numpy as np 
import pandas as pd
import math
import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker
import matplotlib as mlp
from matplotlib import rcParams
import itertools
import matplotlib.font_manager

### Boltzmann, emittance, view factor

In [10]:
epsilon = 1.0
sigma = 5.67*10**(-8) 
alpha = 1.0

Fgnd = 0.5
Fsky = 0.5
Fair = math.sqrt(2)/4 

## Excel file

In [2]:
df_solar_data = pd.read_csv('../data/Solar data.csv')
Tgnd = [df_solar_data["Ground temperature"][i] for i in range(len(df_solar_data["Time"]))]
Tair = [df_solar_data["Temperature"][i] for i in range(len(df_solar_data["Time"]))] 
q_alpha = [(df_solar_data["Insolation"][i])*alpha for i in range(len(df_solar_data["Time"]))]

## Constant

### unit change

In [3]:
# time unit change 
hour_to_sec = 3600
min_to_sec = 60   

### set timestep, duration

In [4]:
t = 20 # [s]   # time step
duration = len(df_solar_data["Time"])*t/hour_to_sec # [h]

## Dictionary

### set material 

In [6]:
concrete_dict =    {"Length"                :  0.6, # [m]
                    "Thermal_conductivity"  :    2, # [W/(m*K)]
                    "Specific_heat"         : 1000, # [J/(Kg*K)]
                    "Density"               : 1000, # [kg/m3] 
                    "Devide"                :    3, # [None]
                     } 
concrete_dict['Volumetric_heat_capcity'] = concrete_dict["Specific_heat"]*concrete_dict["Density"] # 체적열용량은 따로추가 직접 계산 번거로움 감소

insulation_1_dict = {"Length"                :  0.3, # [m]
                     "Thermal_conductivity"  :    2, # [W/(m*K)]
                     "Specific_heat"         : 1000, # [J/(Kg*K)]
                     "Density"               : 1000, # [kg/m3] 
                     "Devide"                :    3, # [None]
                     } 
insulation_1_dict['Volumetric_heat_capcity'] = insulation_1_dict["Specific_heat"]*insulation_1_dict["Density"]

insulation_2_dict = {"Length"                :  0.3, # [m]
                     "Thermal_conductivity"  :    2, # [W/(m*K)]
                     "Specific_heat"         : 1000, # [J/(Kg*K)]
                     "Density"               : 1000, # [kg/m3] 
                     "Devide"                :    3, # [None]
                     } 
insulation_2_dict['Volumetric_heat_capcity'] = insulation_2_dict["Specific_heat"]*insulation_2_dict["Density"]

insulation_3_dict = {"Length"                :  0.3, # [m]
                     "Thermal_conductivity"  :    2, # [W/(m*K)]
                     "Specific_heat"         : 1000, # [J/(Kg*K)]
                     "Density"               : 1000, # [kg/m3]
                     "Devide"                :    3, # [None] 
                     } 
insulation_3_dict['Volumetric_heat_capcity'] = insulation_3_dict["Specific_heat"]*insulation_3_dict["Density"]

## list

In [7]:
t_list = [t*i for i in range(int(duration*hour_to_sec/t))]

## Function

In [8]:
# Degree to Kelvin  
def D2K(val): 
    return val + 273.15 

In [9]:
#                               Material_dict
def get_temp_data(M_dict1, M_dict2, M_dict3, M_dict4, M_dict5):

    ## Constant
    # --------------------------------------------------------------------------------------------
    # 총 셀 노드 개수 변수
    N = M_dict1["Divide"] + M_dict2["Divide"] + M_dict3["Divide"] + M_dict4["Divide"] + M_dict5["Divide"]
    # 초기온도 
    initial_temp = D2K(20) 
    # --------------------------------------------------------------------------------------------
    
    ## 벽체의 열물성을 담은 리스트 정의 [list]
    # --------------------------------------------------------------------------------------------
    for Midx in [M_dict1, M_dict2, M_dict3, M_dict4, M_dict5]: # Material dict를 넣어줌
    #                                                                        Divide 개수 만큼 복붙
        del_x =                   [Midx["length"]/Midx["Divide"]   for _ in range(Midx["Divide"])] 
        k =                       [Midx["Thermal_conductivity"]    for _ in range(Midx["Divide"])]
        c =                       [Midx["Specific_heat"]           for _ in range(Midx["Divide"])]
        rho =                     [Midx["Density"]                 for _ in range(Midx["Divide"])]
        C =                       [Midx["Volumetric_heat_capcity"] for _ in range(Midx["Divide"])]
        # x/k
        K =                       [(Midx["length"]/Midx["Divide"])
                                  /Midx["Thermal_conductivity"]    for _ in range(Midx["Divide"])]
        # k/x
        R =                       [Midx["Thermal_conductivity"]
                                  /(Midx["length"]/Midx["Divide"]) for _ in range(Midx["Divide"])]
    
    # left 
    
    del_x_left = [(del_x[i-1] + del_x[i])/2 for i in range(1,N)] # i=0일 때 i-1 정의 불가
    del_x_left.insert(0,del_x[0]/2) # -- 위 컴프리핸션 포문에서 정의할 수 없으므로 따로 정의

    R_left =     [(del_x[i-1]/2)/k[i-1]
                 +(del_x[i]/2)/k[i]
                 for i in range(1,N)]
    R_left.insert(0,R[0]/2) # -- 위 컴프리핸션 포문에서 정의할 수 없으므로 따로 정의

    K_left = [1/R_left[i] for i in range(N)] # R의 역수로 정의

    # right

    del_x_right = [(del_x[i] + del_x[i+1])/2 for i in range(N-1)] # i= N일 때 i+1 정의 불가 
    del_x_right.append(del_x[N]/2) # -- 위 컴프리핸션 포문에서 정의할 수 없으므로 따로 정의

    R_right =     [(del_x[i]/2)/k[i]
                 +(del_x[i+1]/2)/k[i+1]
                 for i in range(N-1)]
    R_right.append(R[N]/2) # -- 위 컴프리핸션 포문에서 정의할 수 없으므로 따로 정의

    K_right = [1/R_right[i+1] for i in range(N)] # R의 역수로 정의
    # --------------------------------------------------------------------------------------------

    # 온도 정보를 담은 행렬 [matrix]
    # --------------------------------------------------------------------------------------------
    T = []
    T_left = []
    T_right = []

    # 온도 정보 전부 초기온도로 일단 담기 (해당 매트릭스를 포함하는 리스트들은 모두 T가 업데이트 됨에 따라 업데이트 해줘야함)
    for _ in range(len(t_list)):

        T.append([initial_temp for _ in range(N)])
        T_left.append([initial_temp for _ in range(N)])
        T_right.append([initial_temp for _ in range(N)])
    
    # 열전달계수, Tsol 정의 # 온도가 다 초기온도로 들어가있기 때문에 Tridigonal matrix 에서 온도를 업데이트 시키며 같이 업데이트 시켜야함
    # --------------------------------------------------------------------------------------------  
    # 이미 엑셀에 정해져있는 값 (T 행렬이 없음)
    h_c =      [df_solar_data["h_c"][n] for n in range(len(t_list))] 

    #  T가 업데이트 됨에 따라 업데이트가 필요한 값 
    h_ground = [epsilon*sigma*Fgnd*((Tgnd[n])**4-(T[n][0])**4)
                /(Tair[n]-T[n][0]) for n in range(len(t_list))]
    h_air =    [epsilon*sigma*Fair*((Tair[n])**4-(T[n][0])**4)
                /(Tair[n]-T[n][0]) for n in range(len(t_list))]
    h_t = [h_c[n] + h_ground[n] + h_air[n] for n in range(len(t_list))]
    T_sol = [Tair[n]+q_alpha[n]/h_t[n]-T_left[n][0] for n in range(len(t_list))] # T_left[n][0] = T_os

    # q_sol (alpha)


    ## Tridiagonal matrix
    # --------------------------------------------------------------------------------------------
    
    #          | b1 c1  0  0  0 |   | T[n+1][1] |      | g1 |    
    #          | a2 b2 c2  0  0 |   | T[n+1][2] |      | g2 |               
    #          |  0 a3 b3 c3  0 | . | T[n+1][3] |   =  | g3 |      
    #          |  0  0 a4 b4 c4 |   | T[n+1][4] |      | g4 |
    #          |  0  0  0 a5 b5 |   | T[n+1][5] |      | g5 |

    #                  A{matrix}  *   T{vector}     = B{vector}
    #          특정 타임스탭에서 정의했을 때 벡터인 것이지 이것이 누적되어 결국 matrix로 나옴

    # a,b,c [list]
    a_list = [-t*K_left[i] for i in range(N)]
    b_list = [2*del_x[i]*C[i]+t*K_left[i]+t*K_right[i] for i in range(N)]
    c = [-t*K_right[i] for i in range(N)]

    # A matrix -> 미리 알 수 있는 값
    A = np.zeros((N, N)) # 임시로 0값을 가지는 NxN 행렬 만들기

    for idx in range(N-1): # A matrix에 a,b,c 리스트 넣어주기

        A[idx+1][idx] = a_list[idx+1]
        A[idx][idx] = b_list[idx]
        A[N-1][N-1] = b_list[N-1]
        A[idx][idx+1] = c[idx]

    A_inv = np.linalg.inv(A) # 역행렬 미리 만들어두기

    # B matrix -> g 벡터들을 넣어준다 

    for n in range(len(t_list)-1): # 타임스탭 for 문 
        B = [] # 타임스탭마다 새롭게 정의해줘야함
        # B list 정의하기 
        for i in range(N): # node for 문
            B.append([t*K_left[i]*T[n][i-1]+(2*del_x[i]*C[i]-t*K_left[i]-t*K_right[i])*T[n][i]]) # g_i 추가 열벡터로 표현하기 위해 [] 를 씌워줌
        B.insert(0,[2*t*h_t[0]])        # 여기서 부터 아주 중요해짐 기존에 Tridigonal matrix에서 g_1, g_N을 그대로 참조하면 안되고 복사, 대류에 관한 것을 추가해야함
        B.append([])                    # 그렇게 하기 위해선 Excel로 뽑아 놓았던 데이터를 참고해야함

        # T[n+1] 값 구하고 업데이트 필요한 값들 업데이트 
        for i in range(N): # node for 문 # [0] 은 T가 이중 리스트이기 때문에 리스트를 한 겹 벗겨내기 위함 
            T[n+1][i] = np.dot(A_inv, B)[n][0] # tridiagonal matrix 에서 구한 T[n+1] 값을 다음 계산될 T[n] 리스트 값에 정의 
        # 업데이트가 필요한 값들 업데이트
        h_air[n+1] = epsilon*sigma*Fair*((Tair[n])**4-(T[n][0])**4)/(Tair[n]-T[n][0])
        h_ground[n+1] = epsilon*sigma*Fgnd*((Tgnd[n])**4-(T[n][0])**4)/(Tair[n]-T[n][0])
        h_t[n+1] = h_c[n] + h_ground[n] + h_air[n]
        T_sol[n+1] = Tair[n+1]+q_alpha[n+1]/h_t[n+1]-T_left[n+1][0]
        T_left[n+1][0] = T[n][0] + R_left[0]*h_t[n]*(T_sol[n]-T_left[n][0]) # ----- 표면온도를 q1 = q2를 이용해 첫 번째 노드의 온도와 흐르는 열류 q1의 정의로 계산 (T_os 는 열용량이 없으니 열을 그대로 흘려보낸다)
                                                                                  # external |<-------------------------------->| internal
                                                                                  #      o   o    +    +    +    +    +    +    o    o
                                                                                  #   T_BC1 T_os  0    1    2    3    4    5   T_is  T_BC2
                                                                                  #      ---q1---->         q1(T_BC1 to 0)
                                                                                  #          -q2->          q2(T_os  to 0)    

    



SyntaxError: expected ':' (536033692.py, line 120)

In [ ]:
    # -------------------------------------------Tridiagonal matrix------------------------------------------------------------------------
    a_list = [-t*K for i in range(N)]
    a_list[0] = -2*t*K

    b_list = [2*del_x*C + t*K + t*K for i in range(N)]
    b_list[0] = 2*del_x*C + t*(2*K) + t*K
    b_list[N-1] = 2*del_x*C + t*K + t*(2*K)

    c_list = [-t*K for i in range(N)]
    c_list[N-1] = -t*(2*K)

    #                        [cm]
    # external |--1-----3-----5-----7-----9--| internal space
    #          |  +  |  +  |  +  |  +  |  +  |
    #   (T_BC1)|  0     1     2     3     4  |(T_BC2) 
    #                     [node num]         

    ## Tridiagonal matrix algorithm

    #          | b1 c1  0  0  0 |   | T[n+1][1] |      | g1 |    
    #          | a2 b2 c2  0  0 |   | T[n+1][2] |      | g2 |               
    #          |  0 a3 b3 c3  0 | . | T[n+1][3] |   =  | g3 |      
    #          |  0  0 a4 b4 c4 |   | T[n+1][4] |      | g4 |
    #          |  0  0  0 a5 b5 |   | T[n+1][5] |      | g5 |
    #
    #                  A          *       T         =    B

    # N x N 단위행렬 생성
    A = np.identity(N)

    for idx in range(N-1): 
        A[idx+1][idx] = a_list[idx+1]
        A[idx][idx] = b_list[idx]
        A[N-1][N-1] = b_list[N-1]
        A[idx][idx+1] = c_list[idx]

    A_inv = np.linalg.inv(A)

for idx in range(N-1): 
        A[idx+1][idx] = a_list[idx+1]
        A[idx][idx] = b_list[idx]
        A[N-1][N-1] = b_list[N-1]
        A[idx][idx+1] = c_list[idx]

    A_inv = np.linalg.inv(A)
                                
    for tidx in range(t_list_num-1):

        B = []
        for nidx in range(1,N-1):
            B.append([t*K*T[tidx][nidx-1]+(2*del_x*C - t*K - t*K)*T[tidx][nidx] + t*K*T[tidx][nidx+1]])
        # 포문 밖에서 마지막으로 앞뒤에 추가
        B.insert(0,[2*t*(2*K)*T_BC1[tidx]+(2*del_x*C - t*(2*K) - t*K)*T[tidx][0] + t*K*T[tidx][1]])
        B.append([t*K*T[tidx][N-2]+(2*del_x*C - t*K - t*(2*K))*T[tidx][N-1] + 2*t*(2*K)*T_BC2[tidx]])
        
        for nidx in range(N): 
            T[tidx+1][nidx] = np.dot(A_inv, B)[nidx][0]


In [17]:
B = [1, 2, 3]

transpose_B = [B]

transpose_B = list(zip(*transpose_B))
print(transpose_B)

[(1,), (2,), (3,)]


In [30]:
A = [[1,1,1], [1,1,1], [1,1,1]]
B = [[1],[2],[3]]
C = np.dot(A,B)
C

array([[6],
       [6],
       [6]])

In [28]:
A = np.zeros((3, 3))
A

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])